In [38]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
tqdm.pandas()
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV


In [3]:
train_df=pd.read_csv('./Data/train.csv')

In [4]:
train_df.head()

,id,y,var1,var2,var3,t1_2001,t2_2001,t3_2001,t4_2001,t5_2001,...,t106_9170,t107_9170,t108_9170,t109_9170,t110_9170,t111_9170,t112_9170,t113_9170,t114_9170,t115_9170
0,1,113.0,19,A,A,-0.140642,-0.388162,-0.574181,-0.583770,-0.376220,...,0.584130,-0.100278,-0.846640,-1.461938,-1.801008,-1.804553,-1.500023,-0.980571,-0.377526,0.168243
1,2,119.0,14,C,A,0.023754,-0.105455,-0.245025,-0.310670,-0.249384,...,-3.024336,-2.480266,-1.118005,0.181451,0.726820,0.401501,-0.356744,-0.913405,-0.854943,-0.193468
2,3,109.0,13,A,A,0.180818,0.135004,0.001879,-0.134548,-0.185928,...,-0.156851,-0.262902,-0.668436,-1.172372,-1.387328,-1.016077,-0.103130,0.935900,1.550512,1.405350
3,4,110.0,18,A,A,-0.234780,-0.094543,0.193084,0.508519,0.668783,...,-1.358540,-0.028478,0.535346,-0.116384,-1.542395,-2.486781,-1.631185,1.481241,6.015514,10.233163
4,5,123.0,32,A,A,-0.130934,-0.360313,-0.547045,-0.603495,-0.508025,...,-0.825535,-0.392556,0.156361,0.567142,0.756362,0.809336,0.844580,0.876423,0.801686,0.524566


In [6]:
columns=[f'Roi{i}-Roi{j}'  for i in range(1,116) for j in range (i+1,117) ]

In [7]:
def get_corr(row,test=False):
    if test:
        brain = np.array(row[4:])
    else:
        brain = np.array(row[5:])
    brain = brain.reshape((116, 115)).astype(float)
    corr = np.corrcoef(brain)
    flat = corr[np.triu_indices(corr.shape[0], 1)]
    return flat.tolist()


In [8]:
corr=train_df.progress_apply(lambda x :get_corr(x),axis=1)

  0%|          | 1/600 [00:01<12:07,  1.22s/it]C:\Users\Nicola\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\Nicola\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100%|██████████| 600/600 [00:16<00:00, 35.72it/s]


In [12]:
corr_df=pd.DataFrame(np.array(corr.values.tolist()),columns=columns).fillna(0)

In [13]:
first_cols = train_df.iloc[:, [1,2,3,4]]

In [14]:
new_train = first_cols.join(corr_df)

In [16]:
new_train['var2'].replace({'A':0, 'C':1}, inplace = True)
new_train['var3'].replace({'A':0, 'C':1}, inplace = True)

In [17]:
new_train

,y,var1,var2,var3,Roi1-Roi2,Roi1-Roi3,Roi1-Roi4,Roi1-Roi5,Roi1-Roi6,Roi1-Roi7,...,Roi112-Roi113,Roi112-Roi114,Roi112-Roi115,Roi112-Roi116,Roi113-Roi114,Roi113-Roi115,Roi113-Roi116,Roi114-Roi115,Roi114-Roi116,Roi115-Roi116
0,113.0,19,0,0,0.320408,-0.156421,-0.115766,-0.306091,0.034975,-0.147885,...,0.006686,-0.285579,-0.072048,0.411215,0.322524,0.208173,-0.071341,0.282784,-0.055727,0.144371
1,119.0,14,1,0,0.386406,0.084164,-0.226632,-0.136301,-0.428097,-0.126119,...,0.736991,0.147084,-0.159465,0.264405,0.006974,0.009238,0.266661,0.047870,0.112745,0.142081
2,109.0,13,0,0,0.809958,0.096828,-0.012295,-0.658145,-0.565232,0.436640,...,0.200037,0.393775,-0.085829,0.266476,0.340162,-0.142864,0.342046,0.374683,0.281569,-0.166943
3,110.0,18,0,0,0.371073,0.340416,0.165785,0.319523,0.071653,-0.006067,...,0.711573,0.449607,-0.101063,0.494471,0.630120,0.089310,0.583527,0.391849,0.529610,0.202928
4,123.0,32,0,0,0.238808,-0.075542,-0.312048,0.343195,-0.017375,0.279959,...,0.464747,0.110613,0.242850,0.180382,0.264813,0.369938,0.449401,-0.011589,0.637432,0.323892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,110.0,41,0,0,0.239704,0.334075,-0.133219,0.396904,0.218766,0.401353,...,0.868837,0.570910,0.411275,0.662582,0.576929,0.390116,0.636213,0.414633,0.584703,0.823124
596,120.0,64,0,0,0.082087,0.106322,-0.234501,-0.227719,-0.221337,0.428577,...,0.192933,-0.112865,0.101323,0.343547,0.358325,0.412704,0.412598,0.035520,-0.126774,0.412996
597,96.0,27,0,0,0.550363,-0.173623,-0.087052,-0.076264,-0.334513,0.005763,...,0.673816,0.309224,0.328117,0.451602,0.389896,0.261399,0.427783,0.504431,0.574561,0.499287
598,118.0,29,1,0,0.302419,0.042842,-0.557081,-0.029869,-0.165823,0.216248,...,0.168198,0.348409,-0.134313,0.081805,0.124225,-0.034212,-0.042134,0.136920,0.025787,-0.288578


# Train

## Linear Regession

In [19]:
reg = LinearRegression()

In [20]:
X_train, X_test, y_train, y_test = train_test_split(new_train.drop(columns='y'), new_train['y'], test_size=0.33, random_state=42)

In [21]:
reg.fit(X_train,y_train)

LinearRegression()

In [22]:
y_pred=reg.predict(X_test)

In [25]:
mean_squared_error(y_test, y_pred,squared=False)

15.923881311476057

## SVR

In [30]:
params = {'kernel' : ('linear', 'poly', 'rbf', 'sigmoid'),
        'C' : [1,5,10],
        'degree' : [3,8],
        'coef0' : [0.01,10,0.5],
        'gamma' : ('auto','scale')},


In [33]:
svr_reg=SVR()
grids = RandomizedSearchCV(svr_reg,params,cv=3,verbose=5,n_iter=30,n_jobs=-1)
grids.fit(X_train,y_train)


Fitting 3 folds for each of 30 candidates, totalling 90 fits


RandomizedSearchCV(cv=3, estimator=SVR(), n_iter=30, n_jobs=-1,
                   param_distributions=({'C': [1, 5, 10],
                                         'coef0': [0.01, 10, 0.5],
                                         'degree': [3, 8],
                                         'gamma': ('auto', 'scale'),
                                         'kernel': ('linear', 'poly', 'rbf',
                                                    'sigmoid')},),
                   verbose=5)

In [37]:
grids.best_params_

{'kernel': 'rbf', 'gamma': 'scale', 'degree': 3, 'coef0': 10, 'C': 10}

In [34]:
y_pred=grids.predict(X_test)

In [35]:
mean_squared_error(y_test, y_pred,squared=False)

15.051786941609702

In [167]:
#retrain on all dataset
svr_reg.fit(new_df.drop(columns='y'),new_df['y'])

SVR(degree=1, epsilon=0.2, kernel='poly')

# Test 

In [150]:
test_df=pd.read_csv('./Data/test.csv')

In [152]:
test_df

,id,var1,var2,var3,t1_2001,t2_2001,t3_2001,t4_2001,t5_2001,t6_2001,...,t106_9170,t107_9170,t108_9170,t109_9170,t110_9170,t111_9170,t112_9170,t113_9170,t114_9170,t115_9170
0,17,32,A,A,-0.350440,-0.222368,0.023356,0.243820,0.307884,0.181462,...,-0.249206,-0.299956,-0.230605,-0.002176,0.307232,0.574038,0.719488,0.739103,0.657216,0.466077
1,19,11,C,A,-0.637826,-0.626851,-0.360784,0.030366,0.336757,0.405507,...,0.747080,1.451542,1.888522,1.843617,1.353949,0.721757,0.341038,0.461075,1.052172,1.853388
2,25,17,A,C,-0.012941,-0.104797,-0.236863,-0.323248,-0.289526,-0.127867,...,-2.263840,-2.082246,-1.299818,-0.087112,1.200546,2.217524,2.777598,2.863644,2.528300,1.804936
3,32,17,A,C,0.090513,0.210141,0.258353,0.176808,-0.021250,-0.246477,...,-2.662342,-1.246634,0.756969,2.115026,2.007031,0.606600,-1.034530,-1.731524,-1.012911,0.541681
4,33,16,A,C,0.020757,0.004422,0.048168,0.217449,0.499191,0.774152,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,777,20,A,C,0.196767,0.279578,0.269504,0.142366,-0.050581,-0.192273,...,1.501500,0.315651,-0.438466,-0.390447,0.203565,1.092143,1.984272,2.303799,1.587597,0.184648
195,783,24,A,C,0.166928,0.062544,-0.064816,-0.223924,-0.397589,-0.477872,...,0.998999,2.150780,1.765160,0.167519,-1.398137,-2.035133,-1.780128,-1.156148,-0.511489,0.100604
196,784,24,A,C,0.467909,0.601598,0.466428,0.064870,-0.453154,-0.874220,...,4.963199,1.800244,-2.997877,-5.256964,-2.305884,3.395893,5.967640,2.081252,-5.207866,-9.263192
197,792,29,C,C,0.158687,0.187958,0.187666,0.168700,0.137513,0.109296,...,-2.660509,-1.084989,0.932748,2.014907,1.678764,0.401733,-0.877058,-1.297012,-0.560106,0.733061


In [157]:
corr_test=test_df.progress_apply(lambda row:get_corr(row,test=True),axis=1)

  1%|          | 1/199 [00:00<01:38,  2.01it/s]C:\Users\Nicola\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\Nicola\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100%|██████████| 199/199 [00:05<00:00, 35.95it/s]


In [158]:
test_final_df=pd.DataFrame(np.array(corr_test.values.tolist()),columns=columns)

In [159]:
test_final_df

,ROI1-ROI2,ROI1-ROI3,ROI1-ROI4,ROI1-ROI5,ROI1-ROI6,ROI1-ROI7,ROI1-ROI8,ROI1-ROI9,ROI1-ROI10,ROI1-ROI11,...,ROI112-ROI113,ROI112-ROI114,ROI112-ROI115,ROI112-ROI116,ROI113-ROI114,ROI113-ROI115,ROI113-ROI116,ROI114-ROI115,ROI114-ROI116,ROI115-ROI116
0,0.574820,-0.184674,0.335784,-0.433897,-0.314960,0.121238,-0.304749,-0.360175,-0.213842,0.592588,...,0.188771,0.237508,0.273457,0.225723,-0.060189,0.135940,-0.068397,0.239504,0.221618,-0.186755
1,0.459187,0.091974,-0.012738,0.018270,-0.144327,0.286588,-0.024430,0.003806,-0.102631,-0.192910,...,0.395746,-0.245494,0.173441,0.203784,-0.069734,0.214056,0.230202,0.037493,0.031108,0.125471
2,0.318010,0.118248,-0.275287,0.067088,0.031507,-0.158764,-0.122662,0.233291,0.070906,0.441458,...,0.656703,0.428289,0.123900,0.090384,0.504827,0.382075,0.136943,0.402817,0.237602,0.293227
3,0.405577,-0.424435,-0.260486,-0.053301,0.118364,-0.187334,-0.161422,-0.307382,-0.090365,0.439814,...,0.560449,0.255396,-0.304971,-0.070677,0.186580,-0.116094,0.077223,0.041814,-0.004182,0.266954
4,0.531680,0.145877,-0.131794,0.361544,0.015393,0.055204,-0.210112,0.204085,0.027663,0.721721,...,0.003357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,0.217994,0.204179,-0.118850,0.250223,0.456717,-0.195145,-0.325952,-0.053743,-0.012564,0.794479,...,0.218118,0.021160,0.018531,0.039184,-0.006917,-0.157338,-0.094451,0.751545,0.652086,0.765714
195,0.465376,0.038288,-0.065834,-0.123540,-0.223827,0.372467,-0.039882,-0.101934,-0.115844,0.628183,...,0.453787,0.121769,0.407962,0.322393,0.502507,0.234785,0.260871,0.272752,0.113825,0.565614
196,0.320433,-0.058686,-0.264435,-0.292921,-0.012648,-0.363793,-0.426708,0.040388,-0.286963,0.561433,...,0.072454,-0.090642,0.058275,0.047026,0.451872,0.040126,0.278858,0.593402,0.321660,0.652692
197,0.550265,-0.386250,-0.283560,0.059427,0.095853,-0.092002,0.283123,0.147292,-0.060154,0.400976,...,0.114647,0.535039,0.289765,0.529971,0.317849,0.188834,-0.064073,0.547437,0.426339,0.495522


In [160]:
test_final_df=test_final_df.fillna(0)

In [162]:
test_final_df['var1']=test_df['var1']

In [163]:
test_final_df

,ROI1-ROI2,ROI1-ROI3,ROI1-ROI4,ROI1-ROI5,ROI1-ROI6,ROI1-ROI7,ROI1-ROI8,ROI1-ROI9,ROI1-ROI10,ROI1-ROI11,...,ROI112-ROI114,ROI112-ROI115,ROI112-ROI116,ROI113-ROI114,ROI113-ROI115,ROI113-ROI116,ROI114-ROI115,ROI114-ROI116,ROI115-ROI116,var1
0,0.574820,-0.184674,0.335784,-0.433897,-0.314960,0.121238,-0.304749,-0.360175,-0.213842,0.592588,...,0.237508,0.273457,0.225723,-0.060189,0.135940,-0.068397,0.239504,0.221618,-0.186755,32
1,0.459187,0.091974,-0.012738,0.018270,-0.144327,0.286588,-0.024430,0.003806,-0.102631,-0.192910,...,-0.245494,0.173441,0.203784,-0.069734,0.214056,0.230202,0.037493,0.031108,0.125471,11
2,0.318010,0.118248,-0.275287,0.067088,0.031507,-0.158764,-0.122662,0.233291,0.070906,0.441458,...,0.428289,0.123900,0.090384,0.504827,0.382075,0.136943,0.402817,0.237602,0.293227,17
3,0.405577,-0.424435,-0.260486,-0.053301,0.118364,-0.187334,-0.161422,-0.307382,-0.090365,0.439814,...,0.255396,-0.304971,-0.070677,0.186580,-0.116094,0.077223,0.041814,-0.004182,0.266954,17
4,0.531680,0.145877,-0.131794,0.361544,0.015393,0.055204,-0.210112,0.204085,0.027663,0.721721,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,0.217994,0.204179,-0.118850,0.250223,0.456717,-0.195145,-0.325952,-0.053743,-0.012564,0.794479,...,0.021160,0.018531,0.039184,-0.006917,-0.157338,-0.094451,0.751545,0.652086,0.765714,20
195,0.465376,0.038288,-0.065834,-0.123540,-0.223827,0.372467,-0.039882,-0.101934,-0.115844,0.628183,...,0.121769,0.407962,0.322393,0.502507,0.234785,0.260871,0.272752,0.113825,0.565614,24
196,0.320433,-0.058686,-0.264435,-0.292921,-0.012648,-0.363793,-0.426708,0.040388,-0.286963,0.561433,...,-0.090642,0.058275,0.047026,0.451872,0.040126,0.278858,0.593402,0.321660,0.652692,24
197,0.550265,-0.386250,-0.283560,0.059427,0.095853,-0.092002,0.283123,0.147292,-0.060154,0.400976,...,0.535039,0.289765,0.529971,0.317849,0.188834,-0.064073,0.547437,0.426339,0.495522,29


In [170]:
y_pred_test=svr_reg.predict(test_final_df)

In [172]:
y_pred_test

array([112.48524541, 107.9742695 , 109.28836002, 109.27406428,
       109.25129428, 108.00585667, 108.36915404, 109.53978185,
       109.53488903, 108.27682714, 109.12666967, 107.24689681,
       109.86064948, 107.72338198, 108.78559092, 109.03074489,
       109.39470834, 109.49425655, 109.60327633, 108.23951968,
       109.16758611, 109.05039471, 109.78752054, 109.26719304,
       108.31139636, 108.70787101, 108.51679156, 108.97712267,
       110.21838906, 108.95709755, 108.33310928, 108.99406531,
       109.08108765, 109.73369887, 109.43799765, 109.59982365,
       108.34789475, 108.82990657, 109.08337224, 107.10795249,
       109.52910143, 107.94646969, 108.47896635, 108.21659796,
       107.94349594, 109.08501064, 109.29966473, 108.2068465 ,
       108.71662708, 106.98501041, 106.76632186, 108.57347915,
       108.71688769, 107.92803289, 109.08033039, 107.1841864 ,
       109.75735994, 107.94600419, 108.83932747, 108.00048571,
       109.72407354, 108.41892449, 109.83822764, 109.03

In [173]:
df_leaderboard=pd.DataFrame({'id':test_df['id'],'target':y_pred_test})

In [174]:
df_leaderboard

,id,target
0,17,112.485245
1,19,107.974269
2,25,109.288360
3,32,109.274064
4,33,109.251294
...,...,...
194,777,110.774817
195,783,110.402434
196,784,110.895852
197,792,112.362893


In [176]:
df_leaderboard.to_csv('G14_01',index=False)